In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Modules Installation**

In [ ]:
!pip install -q twarc
!pip install -q jsonlines

     |████████████████████████████████| 51kB 5.9MB/s 


## **Import Modules**

In [ ]:
import os,zipfile,glob
import pandas as pd
import twarc

## **Total IEEE Dataset**

In [ ]:
path = "/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Tweet_CSV_File/*.csv"
csv_list = glob.glob(path) # collecting all files  same path 
print(len(csv_list))

61


In [ ]:
data = pd.DataFrame()
for f in csv_list:
  data1 = pd.read_csv(f,header=None)#reading the csv file
  data = pd.concat([data,data1],ignore_index=True)#concating the two data frames
  data.reset_index(drop=True,inplace=True) #resetting the indexes

In [ ]:
data.rename({0:'tweetID',1:'sentiment_score'},axis=1,inplace=True)#renaming the indexes

In [ ]:
data.to_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Tweet_Dataset/data.csv',index=False)#converting the dataframe to CSV

In [ ]:
for f in csv_list:
  df = pd.read_csv(f,header=None) #reading the CSV file
  df = df[0] #Only taking the Tweet Id's from the dataset
  base = os.path.basename(f) #returning the name of the file
  path = '/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Tweets_ID/'+base
  df.to_csv(path,index=False) # converting the dataframe to CSV

## **Hydrating Tweets**

In [ ]:
# Insert API Keys here { run : "auto"}

# These keys are received after applying for a twitter developer account

consumer_key = "8AO6OU5ubyi4XO47b1C7Sjdlz"
consumer_secret = "FS1usPrfPolvjLXbwGka5N8TWkOZhUsdxGmmTwuO016koesUSt"
access_token = "1151573806680592384-OUFeUtpsRFZM6jQxl1AG99NEjlY0Kt"
access_token_secret = "KKHmkHkDGVaDof8XK4fKKI52DmNl4vZlaXnx85WRfd4Lr"

t = twarc.Twarc(consumer_key, consumer_secret, access_token, access_token_secret) #Initializing Twarc

In [ ]:
import jsonlines, json

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Geolocation_Dataset/april28-june18.csv',header=None) #Loading IEEE Geodata
data = data[0] #Taking only Tweet IDs for Hydration
ids = data.values.tolist() #Getting list of tweet ids from pandas DataFrame
hydrated_tweets = [] #Creating empty list
ids_to_hydrate = set(ids) #Creating ids_to_hydrate list

# Now, use twarc and start hydrating
for tweet in t.hydrate(ids_to_hydrate): #Initializing Hydrate Iterator with twarc
  if tweet['place'] != None: #Checking for place value
    if tweet['place']['country'] == 'India': #Checking if Country is India
      hydrated_tweets.append(tweet) #Appending Obtained tweet to hydrated_tweets list

In [ ]:
output_filename = "/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Geolocation_Dataset/output.csv" #output file path

In [ ]:
# Convert jsonl to csv
import csv, jsonlines

# These are the column name that are selected to be stored in the csv
keyset = ["created_at", "id", "id_str", "full_text", "source", "truncated", "in_reply_to_status_id",
          "in_reply_to_status_id_str", "in_reply_to_user_id", "in_reply_to_user_id_str", 
          "in_reply_to_screen_name", "user", "coordinates", "place", "quoted_status_id",
          "quoted_status_id_str", "is_quote_status", "quoted_status", "retweeted_status", 
          "quote_count", "reply_count", "retweet_count", "favorite_count", "entities", 
          "extended_entities", "favorited", "retweeted", "possibly_sensitive", "filter_level", 
          "lang", "matching_rules", "current_user_retweet", "scopes", "withheld_copyright", 
          "withheld_in_countries", "withheld_scope", "geo", "contributors", "display_text_range",
          "quoted_status_permalink"]

# Writes them out (Saving output CSV file with the Indian COVID-19 Tweets)
with  open(output_filename, "w+") as output_file:
    d = csv.DictWriter(output_file, keyset)
    d.writeheader()
    d.writerows(hydrated_tweets)

In [ ]:
import pandas as pd

data = pd.read_csv(output_filename)# reading the file
data1 = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Geolocation_Dataset/april28-june18.csv',header=None)
data1.rename({0:'id',1:'Sentiment'},inplace=True,axis=1)#renaming the columns
result = pd.merge(data,data1,on='id')#merging the two dataframes
result.drop_duplicates(subset ="id", keep = False, inplace = True)#removing duplicates
result.to_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Tweet_Dataset/India_Dataset.csv',index=False)# Converting to CSV

## **Sentiment Addition**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Tweet_Dataset/hydrated_corona_tweets_01")
df = df[['id','retweet_count','lang','text']]#selection of Features
data = df[df["lang"]=="en"] #Extracting the tweets from dataframe whose language is English

In [ ]:
data

,id,retweet_count,lang,text
0,1240727821028405249,631,en,RT @ebs_the_great: A lot of you would actually...
1,1240727810249043969,2568,en,RT @Damii_aros: Mayorkun is somewhere in the s...
2,1240727817555447808,1004,en,RT @iam_erhnehst: Everything was fine in the w...
3,1240727819119968256,12631,en,RT @chandlerriggs: here’s a deleted scene from...
4,1240727808617230336,159,en,RT @RealSaavedra: Good.\n\nIt came from China.
...,...,...,...,...
602817,1240861451239051265,0,en,"Hey governor, I found your 56% percent https:/..."
602818,1240861453311070209,0,en,A Doctor's Heart-touching Appeal on Corona : ...
602819,1240861449263763456,9780,en,RT @Getthebagcoach: Dawg y’all funny man..... ...
602820,1240861452497571845,0,en,LMAOOO I hate this 😭😭😭 https://t.co/aJJV9SxCOz


In [ ]:
tweet_ids_file = "/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Tweet_CSV_File/corona_tweets_01.csv" #Reference Original CSV Path

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Tweet_Dataset/hydrated_corona_tweets_01') # reading the file
data.rename({'text':'full_text'},inplace=True,axis=1)
dataset = data[['id','full_text','retweet_count']]
df = pd.read_csv(tweet_ids_file,header=None)
df.rename({0:'id',1:'Sentiment'},inplace=True,axis=1)#renaming the columns
output = pd.merge(dataset,df,on='id')#merging (inner) the two dataframes
output.drop_duplicates(subset ="id", keep = False, inplace = True)#removing duplicates
output.to_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Tweet_Dataset/Dataset.csv',index=False)# Converting to CSV

In [ ]:
dd = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Tweet_Dataset/Dataset.csv')

In [ ]:
dd.head()

,id,full_text,retweet_count,Sentiment
0,1240727817555447808,RT @iam_erhnehst: Everything was fine in the w...,1004,0.416667
1,1240727819119968256,RT @chandlerriggs: here’s a deleted scene from...,12631,0.357143
2,1240727808617230336,RT @RealSaavedra: Good.\n\nIt came from China.,159,0.700000
3,1240727812610437120,RT @LiILevi: The only man who can save us from...,7073,0.000000
4,1240727813008666624,RT @askjillian: Disgusting. But let’s clutch o...,5,-0.566667


In [ ]:
# Function to Convert the IEEE Sentiment score to Sentiment Text namely positive, negative, neutral
def func(x):
    if x < 0:
        return "negative"
    elif x == 0:
        return "neutral"
    else:
        return "positive"

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Tweet_Dataset/India_Dataset.csv")# reading the file
y= df1.Sentiment
X= df1.drop('Sentiment',axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.75,random_state=111)#Spliting the dataset 
df2 = pd.concat([X_train,y_train],axis=1)#concating the two datasets
df3 = pd.concat([X_test,y_test],axis=1)#concating the two datasets
df3.reset_index(drop=True,inplace=True)#resetting the indexes
df3['Sentiment'] = df3['Sentiment'].apply(lambda x : func(x))#converting sentiment score to sentiment text
df3.to_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Final_Dataset/Test_India.csv',index=False)# Converting to CSV

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Tweet_Dataset/Dataset.csv')# reading the file
df4=data.sample(n=58944,random_state=123)# sampling the dataset
df5=pd.concat([df2,df4],ignore_index=True)#Concating the two datasets 
df5.reset_index(drop=True,inplace=True)# restting the indexes
df5['Sentiment'] = df5['Sentiment'].apply(lambda x : func(x))#converting sentiment score to sentiment text
df5.to_csv("/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Final_Dataset/Data.csv",index=False)# Converting to CSV

## **Splitting Final Data to Train and Test sets for DL Model**

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Final_Dataset/Data.csv')# reading the file
y = data.Sentiment
X = data.drop('Sentiment',axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=123)
df1 = pd.concat([X_train,y_train],axis=1)#concating the two datasets
df1.reset_index(drop=True,inplace=True)#resetting the indexes
df1.to_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Final_Dataset/Train_Data.csv',index=False) # Saving Training/Validation data for DL Model
df2 = pd.concat([X_test,y_test],axis=1)#concating the two datasets
df2.reset_index(drop=True,inplace=True)#resetting the indexes
df2.to_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Final_Dataset/Test_Data.csv',index=False)# Saving Test data for DL Model

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,4,5,10,11,12,13,16,17,23,24,25,26,27,29,36,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# **Data Cleaning**

In [ ]:
#Defining functions for Cleaning and Normalization of Data
import re
import string
def replace_url(string): # cleaning of URL
    text = re.sub(r'http\S+', 'LINK', string)
    return text


def replace_email(text):#Cleaning of Email related text
    line = re.sub(r'[\w\.-]+@[\w\.-]+','MAIL',str(text))
    return "".join(line)

def rep(text):#cleaning of non standard words
    grp = text.group(0)
    if len(grp) > 3:
        return grp[0:2]
    else:
        return grp# can change the value here on repetition
def unique_char(rep,sentence):
    convert = re.sub(r'(\w)\1+', rep, sentence) 
    return convert

def find_dollar(text):#Finding the dollar sign in the text
    line=re.sub(r'\$\d+(?:\.\d+)?','PRICE',text)
    return "".join(line)

def replace_emoji(text):
    emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F" # emoticons
    u"\U0001F300-\U0001F5FF" # symbols & pictographs
    u"\U0001F680-\U0001F6FF" # transport & map symbols
    u"\U0001F1E0-\U0001F1FF" # flags (iOS)
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'EMOJI', text)

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']',
          '>', '%', '=', '#', '*', '+', '\\', '•', '~', '@', '£', '·', '_', '{', '}', '©', '^',
          '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â', '█',
          '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶',
          '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼',
          '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
          'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪',
          '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']

def clean_text(text: str) -> str:
    text = str(text)
    for punct in puncts + list(string.punctuation):
        if punct in text:
            text = text.replace(punct, f'')
    return text
   
def replace_asterisk(text):
    text = re.sub("\*", 'ABUSE ', text)
    return text

def remove_duplicates(text):
    text = re.sub(r'\b(\w+\s*)\1{1,}', '\\1', text)
    return text

def change(text):
    if(text == ''):
        return text
  #calling the subfunctions in the cleaning function
    text = replace_email(text)
    text = replace_url(text)
    text = unique_char(rep,text)
    text = replace_asterisk(text)
    text = remove_duplicates(text)
    text = clean_text(text)
    return text

In [ ]:
import pandas as pd

# Loading different csv files for cleaning 
pathname = "/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Final_Dataset/Data.csv"

Data = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Final_Dataset/Data.csv') #reading the file
Data['full_text'] = Data['full_text'].apply(lambda x : change(x)) # Running cleaning and normalization function on datasets
Data.to_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Cleaned_Dataset/Data.csv') #converting to CSV

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,4,5,10,11,12,13,16,17,23,24,25,26,27,29,36,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
dd1 = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Cleaned_Dataset/Data.csv')
dd1.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,5,6,11,12,13,14,17,18,24,25,26,27,28,30,37,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,created_at,id,id_str,full_text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,coordinates,place,quoted_status_id,quoted_status_id_str,is_quote_status,quoted_status,retweeted_status,quote_count,reply_count,retweet_count,favorite_count,entities,extended_entities,favorited,retweeted,possibly_sensitive,filter_level,lang,matching_rules,current_user_retweet,scopes,withheld_copyright,withheld_in_countries,withheld_scope,geo,contributors,display_text_range,quoted_status_permalink,Sentiment
0,0,Mon May 18 08:55:05 +0000 2020,1262305696126849025,1.262306e+18,Wows theres fake what the hell are you hard on...,"<a href=""http://instagram.com"" rel=""nofollow"">...",False,NaN,NaN,NaN,NaN,NaN,"{'id': 2593245356, 'id_str': '2593245356', 'na...","{'type': 'Point', 'coordinates': [72.8764, 19....","{'id': '7929cea6bd5b32bd', 'url': 'https://api...",NaN,NaN,False,NaN,NaN,NaN,NaN,0,0.0,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [19.0627, 72....",NaN,"[0, 239]",NaN,negative
1,1,Wed May 06 06:58:59 +0000 2020,1257927824234733568,1.257928e+18,In the wake of Coronavirus outbreak the Odisha...,"<a href=""http://instagram.com"" rel=""nofollow"">...",False,NaN,NaN,NaN,NaN,NaN,"{'id': 1029607015990083584, 'id_str': '1029607...","{'type': 'Point', 'coordinates': [77.31974, 28...","{'id': '2e6064382c71b343', 'url': 'https://api...",NaN,NaN,False,NaN,NaN,NaN,NaN,0,2.0,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [28.58052, 77...",NaN,"[0, 235]",NaN,positive
2,2,Sun May 17 08:43:32 +0000 2020,1261940402984288256,1.261940e+18,morning everyone Happy Corona Sunday \nbreakf...,"<a href=""http://instagram.com"" rel=""nofollow"">...",False,NaN,NaN,NaN,NaN,NaN,"{'id': 73610017, 'id_str': '73610017', 'name':...","{'type': 'Point', 'coordinates': [73.0147, 19....","{'id': '514d0719e0a80a43', 'url': 'https://api...",NaN,NaN,False,NaN,NaN,NaN,NaN,0,0.0,"{'hashtags': [{'text': 'morning', 'indices': [...",NaN,False,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [19.0403, 73....",NaN,"[0, 134]",NaN,positive
3,3,Tue Jun 09 07:47:28 +0000 2020,1270261215944536064,1.270261e+18,Another Covid19 death reported in Valley Secon...,"<a href=""http://instagram.com"" rel=""nofollow"">...",False,NaN,NaN,NaN,NaN,NaN,"{'id': 178382091, 'id_str': '178382091', 'name...","{'type': 'Point', 'coordinates': [75.22037778,...","{'id': '2a04d50d4dfcd226', 'url': 'https://api...",NaN,NaN,False,NaN,NaN,NaN,NaN,0,0.0,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [33.74568056,...",NaN,"[0, 240]",NaN,negative
4,4,Sat May 30 20:48:48 +0000 2020,1266833965241053191,1.266834e+18,GujaratStcc\nWorld No Tabacco Day \nworldnotab...,"<a href=""http://instagram.com"" rel=""nofollow"">...",False,NaN,NaN,1.264744e+18,1.264744e+18,GujaratStcc,"{'id': 1144882893384105984, 'id_str': '1144882...","{'type': 'Point', 'coordinates': [70.46004295,...","{'id': '51950e9a1add35ed', 'url': 'https://api...",NaN,NaN,False,NaN,NaN,NaN,NaN,0,0.0,"{'hashtags': [{'text': 'worldnotabaccoday', 'i...",NaN,False,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [21.51885832,...",NaN,"[0, 237]",NaN,neutral


In [ ]:
pathname = "/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Final_Dataset/Test_India.csv"

Data = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Final_Dataset/Test_India.csv') #reading the file
Data['full_text'] = Data['full_text'].apply(lambda x : change(x)) # Running cleaning and normalization function on datasets
Data.to_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Cleaned_Dataset/Test_India.csv') #converting to CSV

In [ ]:
dd2 = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Cleaned_Dataset/Test_India.csv')
dd2.head()

,Unnamed: 0,created_at,id,id_str,full_text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,coordinates,place,quoted_status_id,quoted_status_id_str,is_quote_status,quoted_status,retweeted_status,quote_count,reply_count,retweet_count,favorite_count,entities,extended_entities,favorited,retweeted,possibly_sensitive,filter_level,lang,matching_rules,current_user_retweet,scopes,withheld_copyright,withheld_in_countries,withheld_scope,geo,contributors,display_text_range,quoted_status_permalink,Sentiment
0,0,Sat May 30 13:28:00 +0000 2020,1266723032648814595,1266723032648814595,Corona ke sath jeena padega Dost Dhaval Winds...,"<a href=""http://instagram.com"" rel=""nofollow"">...",False,NaN,NaN,NaN,NaN,NaN,"{'id': 850575572958838784, 'id_str': '85057557...","{'type': 'Point', 'coordinates': [73.92880443,...","{'id': '0a063651d547b2b9', 'url': 'https://api...",NaN,NaN,False,NaN,NaN,NaN,NaN,0,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [18.49371882,...",NaN,"[0, 81]",NaN,neutral
1,1,Mon May 25 11:42:01 +0000 2020,1264884424535158784,1264884424535158784,Follow for more rajanbaba24 \n\nLike Comment ...,"<a href=""http://instagram.com"" rel=""nofollow"">...",False,NaN,NaN,NaN,NaN,NaN,"{'id': 780818039943987202, 'id_str': '78081803...","{'type': 'Point', 'coordinates': [72.83, 21.17]}","{'id': '7a598db14c6554b8', 'url': 'https://api...",NaN,NaN,False,NaN,NaN,NaN,NaN,0,0,"{'hashtags': [{'text': 'entrepreneurbaba24', '...",NaN,False,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [21.17, 72.83]}",NaN,"[0, 238]",NaN,positive
2,2,Thu May 07 08:16:37 +0000 2020,1258309752536182785,1258309752536182785,Iam wandering where the world is going Corona ...,"<a href=""http://instagram.com"" rel=""nofollow"">...",False,NaN,NaN,NaN,NaN,NaN,"{'id': 2946467726, 'id_str': '2946467726', 'na...","{'type': 'Point', 'coordinates': [78.31068706,...","{'id': '7a8b600fe5f170bc', 'url': 'https://api...",NaN,NaN,False,NaN,NaN,NaN,NaN,0,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [17.46493441,...",NaN,"[0, 235]",NaN,neutral
3,3,Mon Jun 01 18:50:30 +0000 2020,1267528967424270342,1267528967424270342,Dr Pankaj Shah aged 65\nM D Physician\nExpired...,"<a href=""http://instagram.com"" rel=""nofollow"">...",False,NaN,NaN,NaN,NaN,NaN,"{'id': 140885006, 'id_str': '140885006', 'name...","{'type': 'Point', 'coordinates': [84.9151306, ...","{'id': '459d5eafbe2e9de0', 'url': 'https://api...",NaN,NaN,False,NaN,NaN,NaN,NaN,1,1,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [26.6516991, ...",NaN,"[0, 162]",NaN,negative
4,4,Sun May 24 12:28:05 +0000 2020,1264533629763563520,1264533629763563520,sppanchariya Keeping prisoners locked up in ja...,"<a href=""https://www.tweetsuite.com/"" rel=""nof...",False,NaN,NaN,3.009124e+09,3.009124e+09,sppanchariya,"{'id': 1198256313081331712, 'id_str': '1198256...","{'type': 'Point', 'coordinates': [77.2810677, ...","{'id': '317fcc4b21a604d5', 'url': 'https://api...",NaN,NaN,False,NaN,NaN,NaN,NaN,0,0,"{'hashtags': [{'text': 'CoronaRiskInJail', 'in...","{'media': [{'id': 1264533626529726464, 'id_str...",False,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [28.6564763, ...",NaN,"[0, 260]",NaN,positive


In [ ]:
pathname = "/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Final_Dataset/Test_Data.csv"

Data = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Final_Dataset/Test_Data.csv') #reading the file
Data['full_text'] = Data['full_text'].apply(lambda x : change(x)) # Running cleaning and normalization function on datasets
Data.to_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Cleaned_Dataset/Test_Data.csv') #converting to CSV

In [ ]:
dd3 = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Cleaned_Dataset/Test_Data.csv')
dd3.head()

,Unnamed: 0,created_at,id,id_str,full_text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,coordinates,place,quoted_status_id,quoted_status_id_str,is_quote_status,quoted_status,retweeted_status,quote_count,reply_count,retweet_count,favorite_count,entities,extended_entities,favorited,retweeted,possibly_sensitive,filter_level,lang,matching_rules,current_user_retweet,scopes,withheld_copyright,withheld_in_countries,withheld_scope,geo,contributors,display_text_range,quoted_status_permalink,Sentiment
0,0,NaN,1240785029669818368,NaN,RT ioproducer There was a nasty flu that went ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative
1,1,NaN,1240758496808722432,NaN,RossMcCaff The venn diagram of people who vote...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative
2,2,NaN,1240797687722774528,NaN,RT LilUziLondon Lil Uzi Vert screaming on live...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,positive
3,3,NaN,1240732917107654656,NaN,the amount of deaths from corona is now the sa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral
4,4,NaN,1240835873400389632,NaN,RT ioproducer There was a nasty flu that went ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative


In [ ]:
pathname = "/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Final_Dataset/Train_Data.csv"

Data = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Final_Dataset/Train_Data.csv') #reading the file
Data['full_text'] = Data['full_text'].apply(lambda x : change(x)) # Running cleaning and normalization function on datasets
Data.to_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Cleaned_Dataset/Train_Data.csv') #converting to CSV

In [ ]:
dd4 = pd.read_csv('/content/drive/MyDrive/Internship Dataset/Data Collection and Preprocessing/Cleaned_Dataset/Train_Data.csv')
dd4.head()

,Unnamed: 0,created_at,id,id_str,full_text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,coordinates,place,quoted_status_id,quoted_status_id_str,is_quote_status,quoted_status,retweeted_status,quote_count,reply_count,retweet_count,favorite_count,entities,extended_entities,favorited,retweeted,possibly_sensitive,filter_level,lang,matching_rules,current_user_retweet,scopes,withheld_copyright,withheld_in_countries,withheld_scope,geo,contributors,display_text_range,quoted_status_permalink,Sentiment
0,0,NaN,1240737070651449344,NaN,JoeBiden China accused the United States of ou...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative
1,1,NaN,1240729619055116288,NaN,RT jhaushofer Please RT A team of researchers ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,positive
2,2,NaN,1240758668494163968,NaN,RT toriarenee This is for CORONA you big fat ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative
3,3,NaN,1240768698815676416,NaN,Bring quarantine on LINK LINK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral
4,4,NaN,1240740130748379136,NaN,RT SPURUrbanist Commuting in the time of coron...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral
